<a href="https://colab.research.google.com/github/creepersguitar/SQL-/blob/main/Database_Tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
from random import choice
from random import sample
import os
import pandas as pd

database_file = 'people_and_food.db'

def delete_db(database_file):
    if os.path.exists(database_file):
        os.remove(database_file)

def init_db(database_file, sql):
    conn = sqlite3.connect(database_file)   
    cursor = conn.cursor()                  
    cursor.executescript(sql)               
    conn.commit()                           
    conn.close()                            

def do_query(database, q):
    con = sqlite3.connect(database)
    cur = con.cursor()
    cur.execute(q)
    rows = cur.fetchall()
    cur.close()
    return(rows)

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file, detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES)
    except Error as e:
        print(e)

    return conn

def execute_sql(db_file, sql_statement):
    conn = create_connection(db_file)
    conn.execute("PRAGMA foreign_keys = 1")
    cur = conn.cursor()
    cur.execute(sql_statement)
    conn.commit()
    conn.close()
    return cur.lastrowid

create_db_sql = '''
BEGIN TRANSACTION;
CREATE TABLE IF NOT EXISTS "Favourite" (
	"ID"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	"PersonID"	INTEGER,
	"FoodID"	INTEGER,
	FOREIGN KEY("FoodID") REFERENCES "FoodAndBeverage"("ID"),
	FOREIGN KEY("PersonID") REFERENCES "Person"("ID")
);
CREATE TABLE IF NOT EXISTS "FoodAndBeverage" (
	"ID"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	"Food"	TEXT
);
CREATE TABLE IF NOT EXISTS "Person" (
	"ID"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	"FirstName"	TEXT
);
CREATE TABLE IF NOT EXISTS "FavouritesList" (
  "FirstName" TEXT,
  "Food" TEXT
);
COMMIT;
'''

delete_db(database_file)
init_db(database_file=database_file,  sql=create_db_sql)

people = ['Aidan',
 'Aksel',
 'Andrew',
 'Crystal',
 'Freddie',
 'George',
 'Jack',
 'Joel',
 'Jono',
 'Tejay',
 'Josh A',
 'Josh F',
 'Josh P',
 'Josh T',
 'Kayleigh',
 'Mina',
 'Nicolette',
 'Steve']

foods = ['Beer',
 'Cheese On Toast',
 'Chocolate',
 'Fillet Steak',
 'Baked Beans',
 'Toad In The Hole',
 'Tofu Surprise',
 'Chicken Feet',
 'Piiizzzzza',
 'Steak and Kidney Pudding',
 'Wine',
 'Vegan Sausage Roll',
 'Apple',
 'Rabbit']

for person in people:
  personq = f"insert into person (firstname) values ('{person}')"
  execute_sql(database_file, personq)

for food in foods:
  foodq = f"insert into foodandbeverage (food) values ('{food}')"
  execute_sql(database_file, foodq)

favourites = []
for person in people:
  foodsample = sample(foods, 10)
  for food in foodsample:
    favourites.append([person, food])

for fav in favourites:
  personidq = f"select id from person where firstname = '{fav[0]}'"
  foodidq = f"select id from foodandbeverage where food = '{fav[1]}'"
  personid = do_query(database_file, personidq)[0][0]
  foodid = do_query(database_file, foodidq)[0][0]
  insertq = f'insert into favourite (PersonID, FoodID) values ({personid}, {foodid})'
  execute_sql(database_file, insertq)

denorm = []
qq = "select Person.FirstName, FoodAndBeverage.Food from Favourite left join Person, FoodAndBeverage on Favourite.FoodID=FoodAndBeverage.ID and Favourite.PersonID=Person.ID"
favs = do_query(database_file, qq)
for fav in favs:
  denormq = f"insert into FavouritesList (FirstName, Food) values ('{fav[0]}', '{fav[1]}')"
  execute_sql(database_file, denormq)

denorm = pd.read_sql('select * from FavouritesList', f'sqlite:///{database_file}')
denorm.to_csv('denorm.csv')